In [0]:
!pip install -q kaggle
%load_ext tensorboard
import datetime
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [5]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle competitions download -c digit-recognizer

!unzip train.csv.zip
!unzip test.csv.zip

Saving kaggle.json to kaggle.json
 76% 7.00M/9.16M [00:00<00:00, 70.9MB/s]
100% 9.16M/9.16M [00:00<00:00, 84.0MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 100MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 73.4MB/s]
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
df = pd.read_csv('train.csv')
df.head()

X = df.iloc[:,1:]
y = df['label']

In [0]:
X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=10)

In [0]:
X = X.reshape(-1, 28, 28, 1)

In [0]:
model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), padding = 'SAME', activation='relu', input_shape = (28, 28, 1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding = 'SAME', activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [26]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 3, 3, 64)         

In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/model/digit_model.h5', monitor='val_loss', save_best_only=True)

model.fit(X, y, batch_size=128, epochs=200, validation_split=0.2, callbacks=[tensorboard_callback, early_stop_callback, model_checkpoint_callback])



Epoch 1/200
263/263 [==============================] - 35s 132ms/step - loss: 0.6023 - accuracy: 0.8312 - val_loss: 0.1297 - val_accuracy: 0.9606
Epoch 2/200
263/263 [==============================] - 35s 131ms/step - loss: 0.1028 - accuracy: 0.9693 - val_loss: 0.0760 - val_accuracy: 0.9780
Epoch 3/200
263/263 [==============================] - 35s 132ms/step - loss: 0.0690 - accuracy: 0.9795 - val_loss: 0.0527 - val_accuracy: 0.9854
Epoch 4/200
263/263 [==============================] - 34s 130ms/step - loss: 0.0507 - accuracy: 0.9846 - val_loss: 0.0593 - val_accuracy: 0.9811
Epoch 5/200
263/263 [==============================] - 34s 130ms/step - loss: 0.0458 - accuracy: 0.9866 - val_loss: 0.0679 - val_accuracy: 0.9826
Epoch 6/200
263/263 [==============================] - 34s 131ms/step - loss: 0.0327 - accuracy: 0.9898 - val_loss: 0.0487 - val_accuracy: 0.9860
Epoch 7/200
263/263 [==============================] - 35s 132ms/step - loss: 0.0317 - accuracy: 0.9908 - val_loss: 0.0430 -

In [0]:
df_test = pd.read_csv('test.csv')
X_test = np.array(df_test)
X_test = X_test.reshape(-1, 28, 28, 1)

In [0]:
pred = model.predict(X_test)
y_pred = pred.argmax(axis=-1)
res = pd.DataFrame()
res['ImageId'] = list(range(1,28001))
res['Label'] = y_pred
res.to_csv("output_submission.csv", index = False)

In [0]:
%tensorboard --logdir logs/fit